<a href="https://colab.research.google.com/github/tajuar-akash-hub/test_repo_for_colaboration/blob/main/insurance_fraud_main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tajuar-akash-hub/Datasets.git

In [ ]:
import pandas as pd
import numpy as np
insurance_fraud=pd.read_csv('/content/Datasets/Health Insurance Fraud Claims csv file.csv')

insurance_fraud2 = insurance_fraud  #copy of the main

In [ ]:
insurance_fraud.head(1)

# info about the datasets(datatype,null-non_null)

In [ ]:
insurance_fraud.info()

Create A copy

#Check the missing value(null values)

In [ ]:
insurance_fraud.isnull().sum()

In [ ]:
insurance_fraud.head(2)

#Take a description about datasets(min,max,mean)

In [ ]:
insurance_fraud.describe()

# Check number of applicant whose age is less than 18

In [ ]:
age_under_18 = insurance_fraud[insurance_fraud['PatientAge']< 18]
len(age_under_18)


#remove all the rows age_under_18

In [ ]:
insurance_fraud=insurance_fraud.drop(age_under_18.index)

insurance_fraud.shape

# check outliers in incomes and age

### 2.1 using Z-scores

In [ ]:
# using z-scores few more outliers algorithms use
def detect_outliers(data,x):
  outliers=[]
  thresold=1.5  #why not 3?
  mean=np.mean(data[x])
  std=np.std(data[x])
  for i in data[x]:
    z_score=(i-mean)/std
    if np.abs(z_score) > thresold:
      outliers.append(i)
  return outliers


In [ ]:
#outliersb using z-scores with threshold 1.5
print(f"outliers in PatientIncome : {len(detect_outliers(insurance_fraud,'PatientIncome'))}")
print(f"outliers in PatientAge : {len(detect_outliers(insurance_fraud,'PatientAge'))}")
print(f"outliers in claimamount : {len(detect_outliers(insurance_fraud,'ClaimAmount'))}")


###2.2 using IQR

In [ ]:
def iqr_outliers(data,x):
  Q1=np.percentile(data[x],25)
  Q3=np.percentile(data[x],75)
  IQR=Q3-Q1
  lower_bound=Q1 - (1.5 * IQR)
  upper_bound=Q1 + (1.5 * IQR)
  outliers=[]

  for i in data[x]:
    if i< lower_bound or i > upper_bound:
      outliers.append(i)
  return outliers



In [ ]:
## outliers in PatientIncome
income_outliers=len(iqr_outliers(insurance_fraud,"PatientIncome"))
# outliers in ClaimAmount
claim_outliers=len(iqr_outliers(insurance_fraud,"ClaimAmount"))
age_outliers=len(iqr_outliers(insurance_fraud,"PatientAge"))

print(f"outliers in PatientIncome : {income_outliers}")
print(f"outliers in claimamount : {claim_outliers}")
print(f"outliers in PatientAge : {age_outliers}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(color_codes="red")
sns.displot(data=insurance_fraud["PatientAge"]).set(title="Distribution of age",xlabel="Ages")

#remove unwanted columns from the datasets


In [ ]:
insurance_fraud = insurance_fraud[["ClaimDate","ClaimAmount","PatientAge","PatientGender","ProviderSpecialty","ClaimStatus","PatientIncome","PatientMaritalStatus","PatientEmploymentStatus","ProviderLocation","ClaimType","ClaimSubmissionMethod","ClaimLegitimacy"]]

print(f"Shape of the datasets: {insurance_fraud.shape}")

insurance_fraud.head(1)



In [ ]:
insurance_fraud[['PatientAge']]

# all coulmns unique values

In [ ]:
all_unique=insurance_fraud.nunique()
all_unique_df = pd.DataFrame(all_unique, columns=['Unique Value Count'])
all_unique_df

# Check duplicated values

In [ ]:
duplicates = insurance_fraud.duplicated()
flag=0
for duplicate in duplicates:
  if duplicate == True:
    flag=1
    print(f"duplicate values {duplicate}")
if flag == 0:
  print("No duplicates row")



# Copy of Original data

In [ ]:
copy_insurance_fraud=insurance_fraud.copy()

#  Encoding Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
le=LabelEncoder()

# Use Ordinal encoders to encode ClaimStatus

In [ ]:
ordinal_categories=["Denied","Pending","Approved"]

ordinal = OrdinalEncoder(categories=[ordinal_categories])

insurance_fraud["ClaimStatus"]=ordinal.fit_transform(insurance_fraud[["ClaimStatus"]])

insurance_fraud.head()

In [ ]:
categorical_columns = ["PatientGender",
                     "ClaimLegitimacy",
                     "ProviderSpecialty",
                     "PatientMaritalStatus",
                     "PatientEmploymentStatus",
                     "ProviderLocation",
                     "ClaimType",
                     "ClaimSubmissionMethod"
                     ]

for col in categorical_columns:
  insurance_fraud[col]=le.fit_transform(insurance_fraud[col])

insurance_fraud.head()

In [ ]:
insurance_fraud.info()

In [ ]:

Legit_Claim = insurance_fraud["ClaimLegitimacy"].value_counts()

print(f" Value counts of Legit_Claim: {Legit_Claim}")



# OverSampling to Spread the target data equal class

In [ ]:
from imblearn.over_sampling import SMOTE


In [ ]:

# Convert 'ClaimDate' to ordinal

insurance_fraud['ClaimDate'] = pd.to_datetime(insurance_fraud['ClaimDate']).apply(lambda date: date.toordinal())

 #  After converting to datetime, this applies a
 #lambda function to each date in the ClaimDate



#feature and target split

X = insurance_fraud.drop('ClaimLegitimacy',axis=1)  #feature

y = insurance_fraud['ClaimLegitimacy']  #target

smote = SMOTE(random_state=42)

X_resampled , y_resampled = smote.fit_resample(X,y)

oversampled_df = pd.DataFrame(X_resampled , columns = X.columns)

oversampled_df['ClaimLegitimacy'] = y_resampled   #adding a new column n to oversampled_df name 'class' which holds the target

# Shape of new Oversampled_df datasets

In [ ]:
value_counts = oversampled_df.groupby('ClaimLegitimacy').size().reset_index(name = 'Count')

print(value_counts)


In [ ]:
oversampled_df.shape

# EDA Section

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Grouping by Gender and Fraud Status to sum the Claim Amounts
grouped_data = copy_insurance_fraud.groupby(['PatientMaritalStatus', 'ClaimLegitimacy'])['ClaimAmount'].sum().reset_index()

# Plotting
plt.figure(figsize=(10, 6))

# Barplot to show Total Claim Amount vs Fraud for Male and Female
sns.barplot(x='PatientMaritalStatus', y='ClaimAmount', hue='ClaimLegitimacy', data=grouped_data)

# Labels and Title
plt.title('Total Claim Amount vs Fraud Status by Gender', fontsize=16)
plt.xlabel('Gender', fontsize=14)
plt.ylabel('Total Claim Amount', fontsize=14)

# Show plot
plt.show()

# Fradulent activity by Age Range

In [ ]:
copy_insurance_fraud['ClaimLegitimacy'].value_counts()

# Fraud activaity by age

In [ ]:
# Filter only fraudulent claims (ClaimLegitimacy = 0 for fraudulent)

fraudulent_claims =copy_insurance_fraud[copy_insurance_fraud['ClaimLegitimacy'] == "Fraud"]

# Define age ranges (bins) for grouping

age_bins = [0, 20, 40, 60, 80, 100]
age_labels = ['0-20', '21-40', '41-60', '61-80', '81-100']

# Create a new column 'AgeRange' based on the bins

fraudulent_claims['AgeRange'] = pd.cut(fraudulent_claims['PatientAge'], bins=age_bins, labels=age_labels)

# Group by AgeRange and count the number of fraudulent claims
age_fraud_count = fraudulent_claims.groupby('AgeRange').size().reset_index(name='FraudCount')

# Plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x='AgeRange', y='FraudCount', data=age_fraud_count, palette='coolwarm')

plt.title('Fraudulent Activity by Age Range', fontsize=16)
plt.xlabel('Age Range', fontsize=14)
plt.ylabel('Number of Fraudulent Claims', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()


# Gender distribution

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
set
plt.figure(figsize=(6,4))  # Larger figure size for better visibility

# Calculate counts
counts =insurance_fraud['PatientGender'].value_counts()

# Plotting bar chart
sns.barplot(x=counts.index, y=counts.values, palette='Set2')

# Adding title and labels

plt.title("Gender count", fontsize=16)
plt.xlabel("Gender", fontsize=14)
plt.ylabel('Count', fontsize=14)

plt.xticks(rotation=5)  # Rotate x labels for better readability if needed

plt.show()


In [ ]:
insurance_fraud.info()

In [ ]:
copy_insurance_fraud["ClaimType"].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # Added import for seaborn since you're using its color palette

features=["PatientMaritalStatus","PatientEmploymentStatus","ClaimSubmissionMethod","ClaimType"]

# Loop through each feature to generate a pie chart
for f in features:
    plt.figure(figsize=(6,4))  # Set the figure size to 10x8 inches for better visibility

    # Calculate the value counts for the feature, normalize them to percentages
    counts = copy_insurance_fraud[f].value_counts(normalize=True) * 100  # Multiplied by 100 to get percentages

    # Plotting a pie chart with percentage labels on each slice
    plt.pie(counts,
            labels=counts.index,  # Label each slice with the unique values of the feature
            autopct='%1.1f%%',  # Show percentage with 1 decimal place
            startangle=140,  # Rotate the chart for a consistent starting point
            colors=sns.color_palette('Set2'))  # Use a color palette from seaborn

    # Add a title indicating which feature the pie chart represents
    plt.title(f'{f} Distribution', fontsize=16)

    # Ensures the pie chart is a perfect circle
    plt.axis('equal')

    # Display the pie chart
    plt.show()
    print("\n")



In [ ]:
features=["PatientMaritalStatus","PatientEmploymentStatus","ClaimSubmissionMethod","ClaimType"]

for f in features:
  plt.figure(figsize=(6,4))  # Larger figure size for better visibility

# Calculate counts
counts = copy_insurance_fraud[f].value_counts()

# Plotting bar chart
sns.barplot(x=counts.index, y=counts.values, palette='Set2')

# Adding title and labels

plt.title(f'{f} Count', fontsize=16)
plt.xlabel(f'{f}', fontsize=14)
plt.ylabel('Count', fontsize=14)

plt.xticks(rotation=45)  # Rotate x labels for better readability if needed

plt.show()


# Create heatmap to show co-relations between features

In [ ]:
#pearson corelation /heatmap
sns.set(font_scale=2)
plt.subplots(figsize=(30,30))
heat_plot=sns.heatmap(insurance_fraud.corr(method='pearson'),annot=True,cmap='RdYlGn',annot_kws={'size':20})
plt.yticks(fontsize=35)
plt.xticks(fontsize=35)
plt.show()

# Data Spliting (Taget and features variables)

In [ ]:
#import libraries
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
oversampled_df.head()

#Scaling Dataset

In [ ]:

min_max=MinMaxScaler()

#train test split on oversampled_df

X  = oversampled_df.drop('ClaimLegitimacy',axis=1)
                                                                               #Changed_somethings here
y = oversampled_df['ClaimLegitimacy']

X_scaled = min_max.fit_transform(X)

X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)




In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

# Start Applying ML Models ⬇





# RandomForest Classifier





In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_model = RandomForestClassifier(random_state=42)

# random_model.fit(X_train,y_train)

# Train the model
random_model.fit(X_train , y_train)

# Make predictions
y_pred = random_model.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(7,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(random_model, X_train, y_train, cv=5)    #By Akash (experimental)

print(f'Cross-validation scores: {scores}')
print(f'Mean accuracy: {scores.mean() * 100:.2f}%')


# Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
svc = SVC()

# Train the model
svc.fit(X_train, y_train)

# Make predictions
y_pred = svc.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(5,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')                                        #do we need to Scale our data for KNN ( by Akash) ?

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(5,5))
sn.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')

# Logistic Regression (By Akash)

---



In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'l2' , tol = 0.0001 , solver = 'liblinear',
                           max_iter=100,multi_class='auto')

#tol = tolerance , max_itr = max iteration

model.fit(X_train,y_train)

pred_values = model.predict(X_test)


In [ ]:
model.fit(X_train,y_train)

In [ ]:
pred_values = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

# Get predictions (class labels)
pred_values = model.predict(X_test)

# Compute the confusion matrix
cm = confusion_matrix(y_test, pred_values)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)


# Evaluation Metrices

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
accuracy = accuracy_score(y_test,pred_values)

precision = precision_score(y_test,pred_values)

recall = recall_score(y_test,pred_values)

f1 = f1_score(y_test,pred_values)

AUC = roc_auc_score(y_test,pred_values)

In [ ]:
print("accuracy : ",accuracy*100)
print("precision : ",precision*100)
print("recall : ",recall*100)
print("f1 score : ",f1*100)
print("AUC Score : ",AUC*100)

# ROC CURVE

In [ ]:

# Get predicted probabilities
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Probability for the positive class (1)

# Calculate fpr and tpr for the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot the ROC curve
plt.plot(fpr, tpr)
plt.plot([0, 1], ls="--")  # Random guess line

plt.title("ROC Curve for Logistic Regression")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.show()


# Naive Bayes Classifier (By akash)

---



In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#Standard Scaler for Naive Bayes

STD_Scaler = StandardScaler()
STD_Scaled_X =  STD_Scaler.fit_transform(X)
STD_Scaled_X

In [ ]:
#train-test split

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(STD_Scaled_X,y,test_size=0.20)

print("train data size (features):",len(x_train))
print("train data size (target):",len(y_train))

print("test data size(feature)",len(x_test))
print("test data size(target)",len(y_test))



In [ ]:
len(X_train)

In [ ]:
#K-fold validation

from sklearn.naive_bayes import GaussianNB

NB_Classifier = GaussianNB(priors= None , var_smoothing =  1e-09 )    #1e-09 = 1*10^-9

#we add very small value in var_smoothing to avoid 0/0 form

# priors = prior probability  (see the documentation from sklearn)

# var_smoothing = variance smothing where variance = sigma square


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

K_fold = KFold(10)

accuracy = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'accuracy' )

precision = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'precision' )

recall = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'recall' )

f1_score = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'f1' )

AUC = cross_val_score(NB_Classifier , X_train,y_train, cv =K_fold , scoring = 'roc_auc' )

In [ ]:
accuracy
overall_accuracy = sum(accuracy) / len(accuracy)
print(f'Overall Accuracy = {overall_accuracy*100:0.2f}%')


recall
overall_recall = sum(recall) / len(recall)
print(f'overall_recall = {overall_recall*100:0.2f}%')


f1_score

overall_f1_score = sum(f1_score) / len(f1_score)
print(f'overall_f1_score = {overall_f1_score*100:0.2f}%')


overall_AUC = sum(AUC) / len(AUC)
print(f'overall_AUC = {overall_AUC*100:0.2f}%')

In [ ]:
#Applying the Algorithm after k-fold(hyper parameter tuning phase)

NB_Classifier = GaussianNB(priors= None , var_smoothing =  1e-09 )  #tune var_smoothing
NB_Classifier.fit(X_train,y_train)

#unseen data

unseen_prediction = NB_Classifier.predict(X_test)


In [ ]:
#Evaluation metrices

from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score ,roc_curve ,roc_auc_score

accuracy = accuracy_score(unseen_prediction,y_test)

precision = precision_score(unseen_prediction,y_test)

recall = recall_score(unseen_prediction,y_test)

f1 = f1_score(unseen_prediction,y_test)

AUC = roc_auc_score(unseen_prediction,y_test)

print(f'accuracy = {accuracy*100:0.2f}%')
print(f'precision = {precision*100:0.2f}%')
print(f'recall = {recall*100:0.2f}%')
print(f'f1 = {f1*100:0.2f}%')


# Decision Tree Classifier

In [ ]:
len(x_train)


In [ ]:
len(X_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DTregressor=DecisionTreeRegressor(criterion='squared_error',
                                  max_depth= 10,
                                  min_samples_split= 2,
                                  min_samples_leaf= 1,
                                  max_features= None,
                                  max_leaf_nodes= None,
                                  min_impurity_decrease= 0.01
                                  )

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
k_fold = KFold(10)

results1 = cross_val_score(DTregressor, x, y, cv=k_fold , scoring='neg_mean_squared_error')
results2 = cross_val_score(DTregressor, x, y, cv=k_fold , scoring= 'neg_mean_absolute_percentage_error')
results3 = cross_val_score(DTregressor, x, y, cv=k_fold , scoring= 'r2')


In [ ]:
results1

In [ ]:
results2

In [ ]:
overall_MAE_percentage = sum(results2)/len(results2)
print(abs(overall_MAE_percentage)*100)

In [ ]:
overallr2 = sum(results3)/len(results3)
print(overallr2)